
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_3033469/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 19:46:55.772598: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:46:55.842553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 19:46:55.842609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 19:46:55.844167: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 19:46:55.852531: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:46:55.855316: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[40,20]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.02])

Testando combinacao1: Hidden Size=[40, 20], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.5004742879991682
Stopped at epoch:  10
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++


Elapsed time:  2.6062415029991826
Stopped at epoch:  34
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.5209928610001953
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.5364060869997047
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.524901825001507
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.5576858960012032
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.5346026890001667
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 8 ++++++++++++++++++
Elapsed time:  0.6674943899997743
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_4_0,0.802243,0.675968,0.861294,0.735701,0.218859,0.269094,0.025074,0.154261
model_1_4_1,0.837476,0.744094,0.787646,0.776284,0.179866,0.212518,0.038387,0.130574
model_1_4_2,0.864881,0.792318,0.696457,0.799318,0.149537,0.172471,0.054871,0.117130
model_1_4_3,0.886610,0.822546,0.602631,0.808413,0.125489,0.147367,0.071832,0.111821
model_1_4_4,0.908319,0.842797,0.506169,0.809608,0.101463,0.130550,0.089269,0.111124
model_1_4_5,0.927891,0.874479,0.266736,0.798576,0.079803,0.104240,0.132551,0.117563
model_1_4_6,0.929834,0.848155,0.172724,0.765045,0.077653,0.126100,0.149546,0.137133
model_1_4_7,0.930883,0.829420,0.053861,0.733609,0.076492,0.141659,0.171032,0.155482
model_1_4_8,0.931919,0.820157,-0.069535,0.708647,0.075345,0.149351,0.193339,0.170051
model_1_4_9,0.933035,0.816805,-0.180294,0.689979,0.074111,0.152135,0.213360,0.180947


DataFrame salvo em ../results/metrics_1_4
+++++++++++ [1_5] | 1 ++++++++++++++++++
Elapsed time:  0.6572999810014153
Stopped at epoch:  9
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_5] | 2 ++++++++++++++++++
Elapsed time:  0.335800079999899
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_5] | 3 ++++++++++++++++++
Elapsed time:  0.39155445899996266
Stopped at epoch:  6
1/1 [==============================] - 0s 44ms/step
+++++++++++ [1_5] | 4 ++++++++++++++++++
Elapsed time:  0.4010346040013246
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_5] | 5 ++++++++++++++++++
Elapsed time:  0.42669176300114486
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_5] | 6 ++++++++++++++++++
Elapsed time:  0.3387288300000364
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_5] | 7 ++++++++++++++++++
Elapsed time:  0.34673547700

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_5_24,0.895319,0.744971,-0.979979,0.293847,0.115851,0.190204,0.596172,0.381248
model_1_5_23,0.896752,0.746836,-0.947689,0.303687,0.114265,0.188812,0.586449,0.375935
model_1_5_22,0.898415,0.748836,-0.909686,0.315123,0.112424,0.187321,0.575006,0.369761
model_1_5_21,0.900337,0.750937,-0.865135,0.328351,0.110297,0.185754,0.561592,0.362619
model_1_5_20,0.902557,0.753105,-0.812896,0.343647,0.107841,0.184137,0.545863,0.354361
model_1_5_19,0.905103,0.755269,-0.751962,0.361221,0.105023,0.182523,0.527516,0.344872
model_1_5_18,0.907970,0.757351,-0.682229,0.381046,0.101851,0.180970,0.506519,0.334169
model_1_5_17,0.911179,0.759266,-0.602784,0.403296,0.098299,0.179542,0.482598,0.322157
model_1_5_16,0.914659,0.760962,-0.515262,0.427505,0.094448,0.178277,0.456245,0.309086
model_1_5_15,0.918259,0.762487,-0.423537,0.452695,0.090464,0.177139,0.428627,0.295486


...

DataFrame salvo em ../results/metrics_2_7
+++++++++++ [2_8] | 1 ++++++++++++++++++
Elapsed time:  0.303696253000453
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_8] | 2 ++++++++++++++++++
Elapsed time:  0.338262745000975
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [2_8] | 3 ++++++++++++++++++
Elapsed time:  0.34365949099992577
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_8] | 4 ++++++++++++++++++
Elapsed time:  0.29409966799903486
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_8] | 5 ++++++++++++++++++
Elapsed time:  0.31841605200133927
Stopped at epoch:  7
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_8] | 6 ++++++++++++++++++
Elapsed time:  0.5014856130001135
Stopped at epoch:  7
1/1 [==============================] - 0s 18ms/step
+++++++++++ [2_8] | 7 ++++++++++++++++++
Elapsed time:  0.26649897500

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_8_0,0.902347,0.716273,0.971608,0.926387,0.108073,0.063057,0.009271,0.037746
model_2_8_1,0.905585,0.722458,0.966205,0.926187,0.104490,0.061683,0.011035,0.037849
model_2_8_2,0.908127,0.726446,0.951730,0.922764,0.101677,0.060796,0.015761,0.039604
model_2_8_3,0.909480,0.725431,0.923410,0.914044,0.100179,0.061022,0.025009,0.044075
model_2_8_4,0.916304,0.738101,0.941157,0.922271,0.092627,0.058206,0.019214,0.039857
model_2_8_5,0.921005,0.747258,0.936706,0.923038,0.087425,0.056171,0.020667,0.039463
model_2_8_6,0.922219,0.748377,0.911108,0.915623,0.086080,0.055922,0.029026,0.043265
model_2_8_7,0.924958,0.753031,0.909910,0.916333,0.083049,0.054888,0.029417,0.042902
model_2_8_8,0.927343,0.756717,0.903211,0.915171,0.080410,0.054069,0.031604,0.043497
model_2_8_9,0.929879,0.761540,0.901923,0.915892,0.077604,0.052997,0.032025,0.043127


DataFrame salvo em ../results/metrics_2_8
+++++++++++ [2_9] | 1 ++++++++++++++++++
Elapsed time:  0.3375249700002314
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [2_9] | 2 ++++++++++++++++++
Elapsed time:  0.2917647329995816
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_9] | 3 ++++++++++++++++++
Elapsed time:  0.32448049199956586
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 4 ++++++++++++++++++
Elapsed time:  0.30380615099966235
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_9] | 5 ++++++++++++++++++
Elapsed time:  0.32979241399880266
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_9] | 6 ++++++++++++++++++
Elapsed time:  0.3183154319995083
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [2_9] | 7 ++++++++++++++++++
Elapsed time:  0.289724158

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_9_24,0.868547,0.668268,-0.627758,0.537586,0.145480,0.563736,0.349999,0.463154
model_2_9_23,0.869051,0.667767,-0.598708,0.540071,0.144922,0.564587,0.343752,0.460665
model_2_9_22,0.869799,0.668212,-0.568962,0.543476,0.144094,0.563831,0.337356,0.457255
model_2_9_21,0.870517,0.668420,-0.537921,0.546799,0.143299,0.563477,0.330682,0.453926
model_2_9_20,0.871899,0.670965,-0.503723,0.552539,0.141770,0.559153,0.323329,0.448177
model_2_9_19,0.873515,0.674279,-0.467424,0.559182,0.139981,0.553521,0.315524,0.441523
model_2_9_18,0.874836,0.676508,-0.431745,0.564789,0.138519,0.549734,0.307852,0.435907
model_2_9_17,0.875171,0.675154,-0.399758,0.566805,0.138149,0.552034,0.300974,0.433888
model_2_9_16,0.876581,0.677532,-0.361394,0.572816,0.136588,0.547993,0.292725,0.427867
model_2_9_15,0.878564,0.681925,-0.320283,0.580914,0.134394,0.540529,0.283886,0.419756


DataFrame salvo em ../results/metrics_2_9
Testando combinacao3: Hidden Size=[40, 20], regularizer=0.02, learning_rate=0.1
+++++++++++ [3_0] | 1 ++++++++++++++++++
Elapsed time:  1.451634761000605
Stopped at epoch:  19
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_0] | 2 ++++++++++++++++++
Elapsed time:  0.4664337349986454
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_0] | 3 ++++++++++++++++++
Elapsed time:  0.338695854999969
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_0] | 4 ++++++++++++++++++
Elapsed time:  0.3997831899996527
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_0] | 5 ++++++++++++++++++
Elapsed time:  1.771294525000485
Stopped at epoch:  29
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_0] | 6 ++++++++++++++++++
Elapsed time:  1.833680292000281
Stopped at epoch:  28
1/1 [==============================] - 0s 1